<a href="https://colab.research.google.com/github/annigue/mmm_meridian/blob/main/MarketingMixModell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%pip install pyreadr


In [6]:
import pyreadr
import requests
import tempfile

# URL zur .RData-Datei
url = "https://raw.githubusercontent.com/facebookexperimental/Robyn/main/R/data/dt_simulated_weekly.RData"

# Lade Datei in ein temporäres File
response = requests.get(url)
with tempfile.NamedTemporaryFile(suffix=".RData") as tmp_file:
    tmp_file.write(response.content)
    tmp_file.flush()

    # Lese das RData-Objekt
    result = pyreadr.read_r(tmp_file.name)

# Zeige die geladenen Objekte
print(result.keys())

# Extrahiere das DataFrame
df = result["dt_simulated_weekly"]
print(df.head())




odict_keys(['dt_simulated_weekly'])
         DATE       revenue          tv_S     ooh_S       print_S  \
0  2015-11-23  2.754372e+06  22358.346667       0.0  12728.488889   
1  2015-11-30  2.584277e+06  28613.453333       0.0      0.000000   
2  2015-12-07  2.547387e+06      0.000000  132278.4    453.866667   
3  2015-12-14  2.875220e+06  83450.306667       0.0  17680.000000   
4  2015-12-21  2.215953e+06      0.000000  277336.0      0.000000   

     facebook_I  search_clicks_P     search_S  competitor_sales_B  \
0  2.430128e+07         0.000000     0.000000             8125009   
1  5.527033e+06      9837.238486  4133.333333             7901549   
2  1.665159e+07     12044.119653  3786.666667             8300197   
3  1.054977e+07     12268.070319  4253.333333             8122883   
4  2.934090e+06      9467.248023  3613.333333             7105985   

    facebook_S events    newsletter  
0  7607.132915     na  19401.653846  
1  1141.952450     na  14791.000000  
2  4256.375378     n

In [16]:
## Wetter

#%pip install pycountry

from DWD_hist_weather import tagestemp_land, tageswerte_land
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pycountry

# Liste der Bundesländer aus pycountry extrahieren
bundeslaender = [country.name for country in pycountry.subdivisions.get(country_code='DE')]


# Leerer DataFrame für den Gesamtdatensatz
gesamtdatensatz = pd.DataFrame()


# Iteriere über die Bundesländer
for bundesland in bundeslaender:
    pickle_dateiname = f'/content/drive/MyDrive/pickles/{bundesland}.pickle'

    try:
        tageswerte = pickle.load(open(pickle_dateiname, 'rb'))
        print(f'Wetterdaten für {bundesland} aus pickle eingelesen.')
    except (OSError, IOError):
        tageswerte = tageswerte_land(bundesland)  # Funktion tageswerte_land() anpassen oder definieren
        pickle.dump(tageswerte, open(pickle_dateiname, 'wb'))
        print(f'\nWetterdaten für {bundesland} in pickle geschrieben.')


    # Füge eine Spalte mit dem Bundeslandnamen und der Gewichtung hinzu
    tageswerte['Bundesland'] = bundesland

    # Füge die gewichteten Wetterdaten des aktuellen Bundeslandes zum Gesamtdatensatz hinzu
    gesamtdatensatz = pd.concat([gesamtdatensatz, tageswerte])

    # Aufheben der Indexierung, damit die Datumsspalte eine richtige Variable ist
gesamtdatensatz = gesamtdatensatz.reset_index()
gesamtdatensatz.tail()



# Umwandlung der "Datum"-Spalte in das Datumsformat
gesamtdatensatz['Datum'] = pd.to_datetime(gesamtdatensatz['Datum'])

# Ergänzen der Kalenderwoche
gesamtdatensatz['Kalenderwoche'] = gesamtdatensatz['Datum'].dt.isocalendar().week


# Gruppierung nach Bundesland und Kalenderwoche und Berechnung der Aggregationen
wetter_week = gesamtdatensatz.groupby(['Jahr','Kalenderwoche'], as_index=False).agg({
    'TempMean': 'mean',
    'HumidityMean': 'mean',
    'TempMax': 'max',
    'TempMin': 'min',
    'SunshineDuration': 'mean',
    'Jahr': lambda x: x.iloc[0],  # Übernahme des ersten Werts für Jahr (Annahme: gleich für die ganze Woche)
    'Datum': lambda x: x.iloc[0],  # Übernahme des ersten Werts für Datum (Annahme: gleich für die ganze Woche)
    'Monat': lambda x: x.iloc[0],  # Übernahme des ersten Werts für Monat (Annahme: gleich für die ganze Woche)
})

# Die neue Spalte "Product" hinzufügen, die das Produkt der beiden Spalten enthält
wetter_week['Sunshine_Humidity_Interaktion'] = wetter_week['SunshineDuration'] * wetter_week['HumidityMean']
wetter_week['Sunshine_Temp_Interaktion'] = wetter_week['SunshineDuration'] * wetter_week['TempMax']

wetter_week.tail()

column_mapping = {'TempMean': 'seas_TempMean', 'HumidityMean':'seas_HumidityMeand', 'TempMax':'seas_TempMax',
                  'TempMin':'seas_TempMin', 'SunshineDuration': 'seas_SunshineDuration',
                  'Sunshine_Humidity_Interaktion': 'seas_Sunshine_Humidity_Interaktion','Sunshine_Temp_Interaktion':'seas_Sunshine_Temp_Interaktion' }
wetter_week = wetter_week.rename(columns=column_mapping)
#wetter_week.to_excel(r'wetter_week.xlsx', index=False)

#df_seas = pd.read_excel("wetter_week.xlsx")
df_seas = wetter_week
df_seas['Datum'] = pd.to_datetime(df_seas['Datum'], format='%Y-%m-%d')

df_main = pd.merge(df_main, df_seas, on='Datum', how='left')
df_main.tail()

KeyError: 'Bremen'

In [7]:
# Install meridian: from PyPI @ latest release
%pip install --upgrade google-meridian[colab,and-cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.0/886.0 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: natsort
    Found existing installation: natsort 8.4.0
    Uninstalling natsort-8.4.0:
      Successfully uninstalled natsort-8.4.0


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Your runtime has 13.6 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [9]:
#['DATE', 'revenue', 'tv_S', 'ooh_S', 'print_S', 'facebook_I', 'search_clicks_P', 'search_S', 'competitor_sales_B', 'facebook_S', 'events', 'newsletter']

coord_to_columns = load.CoordToColumns(
    time='DATE',
    geo='geo',
    controls=['GQV', 'competitor_sales_B'],
    population='population',
    kpi='revenue',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'Channel0_impression',
        'Channel1_impression',
        'Channel2_impression',
        'Channel3_impression',
        'Channel4_impression',
    ],
    media_spend=[
        'Channel0_spend',
        'Channel1_spend',
        'Channel2_spend',
        'Channel3_spend',
        'Channel4_spend',
    ],
    organic_media=['Organic_channel0_impression'],
    non_media_treatments=['Promo'],
)